In [110]:
new = pd.concat([traindata, testdata], axis=0)

In [111]:
new[new['Anon Student Id']=='88Uhb8760x']['Problem Name']=='EG-FACTOR19'

66162    False
66163    False
66164    False
66165    False
66166    False
         ...  
315      False
316      False
317      False
318      False
319       True
Name: Problem Name, Length: 4839, dtype: bool

In [112]:
new[new['Problem Name']=='EG-FACTOR19']['Anon Student Id']=='88Uhb8760x'

8353      False
8354      False
8355      False
8356      False
8357      False
          ...  
144746    False
144747    False
144748    False
144749    False
319        True
Name: Anon Student Id, Length: 71, dtype: bool

In [117]:
new[144749:144750]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
144749,701925,1bJbgQ32E3,"Unit QUADRATICS-FACTORING_ES, Section QUADRATI...",EG-FACTOR19,1,FinalAnswer,2006-03-28 14:52:59.0,2006-03-28 14:53:21.0,2006-03-28 14:53:21.0,2006-03-28 14:53:21.0,22.0,22.0,NaN,1.0,0.0,0.0,1.0,factor-quadratic-sp,11


In [183]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [112]:
# cols = ['Personal CFAR', 'Problem CFAR', 'Anon Student Id', 'Problem Unit', 'Problem Section',\
#         'Step Name', 'Problem Name', 'Problem View', 'Correct First Attempt', 'KC_num', 'Step CFAR', 'KC CFAR']
cols = ['Personal CFAR', 'Problem CFAR',\
        'Problem View', 'Correct First Attempt', 'KC_num', 'Step CFAR', 'KC CFAR', 'KC CFAR(avg)', 'Problem time', 'Step time']

In [45]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
traindata = pd.read_csv(train_file, sep='\t')
testdata = pd.read_csv(test_file, sep='\t')

In [160]:
student_correct_rate = {}
# CFAR Calculation
for student, group in traindata.groupby(['Anon Student Id']):
    student_correct_rate[student] = (len(group[group['Correct First Attempt'] == 1]), len(group['Correct First Attempt']))
traindata['Personal CFAR'] = traindata['Anon Student Id'].apply(
    lambda x: student_correct_rate[x][0]/student_correct_rate[x][1])
mean_SCFAR = np.mean(
    list(map(lambda x: x[0], list(student_correct_rate.values()))))

problem_correct_rate = {}
problem_time = {}
for problem, group in traindata.groupby(['Problem Name']):
    problem_correct_rate[problem] = 1.0 * len(group[group['Correct First Attempt'] == 1]) / len(group['Correct First Attempt'])
    problem_time[problem] = (group['Step Duration (sec)']).mean() 
    
traindata['Problem CFAR'] = traindata['Problem Name'].apply(
    lambda x: problem_correct_rate[x])
mean_PCFAR = np.mean(list(problem_correct_rate.values()))

traindata['Problem time'] = traindata['Problem Name'].apply(
    lambda x: problem_time[x])
mean_prob_time = np.mean(list(problem_time.values()))
# Seperate
# traindata['Problem Unit'] = traindata['Problem Hierarchy'].str.split(
#     ',', 1).str[0]
# traindata['Problem Section'] = traindata['Problem Hierarchy'].str.split(
#     ',', 1).str[1]

In [162]:
step_correct_rate = {}
step_time = {}
for step, group in traindata.groupby(['Step Name']):
    step_correct_rate[step] = 1.0 * (len(group[group['Correct First Attempt'] == 1]) + 1) / (len(group['Correct First Attempt']) + 2)
    step_time[step] = 1.0 * (group['Step Duration (sec)']).mean() 
                                   
traindata['Step CFAR'] = traindata['Step Name'].apply(lambda x: step_correct_rate[x])
traindata['Step time'] = traindata['Step Name'].apply(lambda x: step_time[x])
mean_STCFAR = np.mean(list(step_correct_rate.values()))
tmp_list = []
for i in step_time.values():
    if not pd.isnull(i):
        tmp_list.append(i)
median_time = np.median(tmp_list)
mean_step_time = np.mean(tmp_list)

In [155]:
print(mean_STCFAR, mean_STIME)
print(traindata[traindata['Step Name'] == '4561.16 = -4371.06(y+7346.41)']['Correct First Attempt'])
for i in step_time.keys():
    if pd.isnull(step_time[i]):
        print(i)

0.6221392479854602 nan
173520    1
Name: Correct First Attempt, dtype: int64
(-1+10)*-6
(-8-6)*-7
(31x-22)/-4 = 28
-0.2 = 85.2(x-81.4)
-20.3 = 8.8y+y+1.9
-3 = -298x/13+12
-3-9 = 5y
-3779.44 = -7663.05(x+5669.58)
-384/47 = 47y/47
-5158.54 = 2458.83(y-3540.29)
-6-8*-4
-67.05 = -96.91(x+78.3)
-7 = -6(x-8)
-7.7 = -6.2(y+3.3)
-8 = -4/x
-9 = (-4x-2)/8
-9 = 10(1x-9)-6x
-x+20-33x = 8
-x+20-46x = -21
0.77612121 = y+61.36
1 = -7x+3
15.7 = -8x
19.1 = 17.3+47.4x-x
25.9y-y-28.5 = -11
3 = -9x-10
3(27y+47)+43y = -13
3.3y-2.4 = 2.7
31 = 20y-y+40
33.9 = y+47.5y-40.4
38.4 = x-9x+22.7
4(2-6)
4+(x/-5) = 8
4+8 = -8+y+8
44.47(y+91.34) = -50.96
45 = 31x+x
4561.16 = -4371.06(y+7346.41)
49+37*5 = y
56Y = -18
6+y/7 = 5
7 = -2/x
8 = -6(x+4)
8.2x-x = 2
81y+141+43y = -13
81y+43y = -154
9/x = 1
9687.01 = 8429.95(x+6521.63)
9x+3 = 5
9y+14y-10 = 4
x = -1-20
x = -13*-25
x+889.18 = 0.37639123


In [156]:
for student, group in traindata.groupby(['Anon Student Id']):
    student_correct_rate[student] = (len(group[group['Correct First Attempt'] == 1]), len(group['Correct First Attempt']))

In [50]:
compound_KCs = set(traindata['KC(Default)'])
KC ={}
KC_correct_rate = {}
for kcs in compound_KCs:
    if pd.isnull(kcs):
        continue
    for kc in kcs.split('~~'):
        KC[kc] = [0,0]
# for KC, group in traindata.groupby(['KC(Default)']):
#     if not pd.isnull(KC):  # KC != 'nan':
#         KC_correct_rate[KC] = 1.0 * len(group[group['Correct First Attempt'] == 1]) / len(
#             group['Correct First Attempt'])
# mean_KCFAR =  np.mean(list(KC_correct_rate.values()))
# traindata['KC CFAR'] = traindata['KC(Default)'].apply(
#     lambda x: KC_correct_rate[x] if not pd.isnull(x) else 0.815)
for _, row in traindata.iterrows():
    if pd.isnull(row['KC(Default)']):
        continue
    else:
        for kc in row['KC(Default)'].split('~~'):
            if row['Correct First Attempt']:
                KC[kc][0] += 1
            KC[kc][1] += 1
KC_correct_rate = {i:KC[i][0]/KC[i][1] for i in KC.keys()}
mean_KCFAR =  np.mean(list(KC_correct_rate.values()))

In [51]:
traindata[0:1]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default),Personal CFAR,Problem CFAR,Step CFAR
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,NaN,1,0,0,1,NaN,NaN,0.748749,0.710197,0.840538


In [74]:
tmp = []
for i, row in traindata.iterrows():
    if i%1000 == 0:
        print(i)
    if pd.isnull(row['KC(Default)']):
        tmp.append(0.815)
    else:
#         for kc in row['KC(Default)'].split('~~'):
        tmp.append(sum([KC_correct_rate[kc] for kc in row['KC(Default)'].split('~~')])\
        / (row['KC(Default)'].count("~")+1))
traindata['KC CFAR(avg)'] = tmp

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [80]:
tmp = []
for i, row in traindata.iterrows():
    if i%1000 == 0:
        print(i)
    if pd.isnull(row['KC(Default)']):
        tmp.append(0.815)
    else:
        tmpp = 1.0
        for kc in row['KC(Default)'].split('~~'):
            tmpp *= KC_correct_rate[kc]
        tmp.append(tmpp)
traindata['KC CFAR'] = tmp

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [118]:
traindata[0:6]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,KC(Default),Opportunity(Default),Personal CFAR,Problem CFAR,Step CFAR,KC CFAR(avg),KC CFAR,KC_num,Problem time,Step time
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,NaN,NaN,0.748749,0.710197,0.840538,0.815000,0.815000,0,40.733454,49.941010
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,NaN,NaN,0.748749,0.710197,0.830609,0.815000,0.815000,0,40.733454,32.949725
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,Define Variable,1,0.748749,0.710197,0.966849,0.966979,0.966979,1,40.733454,10.225832
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,...,"Using small numbers~~Write expression, positiv...",1~~1~~1,0.748749,0.710197,0.404504,0.300044,0.124279,3,40.733454,84.677811
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,...,Entering a given,1,0.748749,0.710197,0.760020,0.800770,0.800770,1,40.733454,34.228587
5,9943,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R5C1,2005-09-09 12:27:30.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,...,Entering a given,2,0.748749,0.710197,0.755473,0.800770,0.800770,1,40.733454,26.553470


In [157]:
#traindata['KC_length'] = traindata['KC(Default)'].astype("str").apply(lambda x: len(x))
traindata['KC_num'] = traindata['KC(Default)'].astype("str").apply(
    lambda x: 0 if x == 'nan' else (x.count('~~') + 1))
# Seperate
# train_x
train_x = traindata[cols].copy()
train_x['Opportunity(Mean)'] = traindata['Opportunity(Default)'].astype(
    "str").apply(lambda x: np.mean(list(map(int, x.replace('nan', '0').split('~~')))))
train_x['Opportunity(Min)'] = traindata['Opportunity(Default)'].astype(
    "str").apply(lambda x: min(list(map(int, x.replace('nan', '0').split('~~')))))

In [158]:
train_x[1:2]

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,KC CFAR(avg),Problem time,Step time,Opportunity(Mean),Opportunity(Min)
1,0.748749,0.710197,1,1,0,0.830609,0.815,0.815,40.733454,32.949725,0.0,0


In [159]:
# Test
# testdata['Problem Unit'] = testdata['Problem Hierarchy'].str.split(
#     ',', 1).str[0]
# testdata['Problem Section'] = testdata['Problem Hierarchy'].str.split(
#     ',', 1).str[1]
testdata['Personal CFAR'] = testdata['Anon Student Id'].apply(
    lambda x: student_correct_rate[x][0]/student_correct_rate[x][1] if x in student_correct_rate.keys() else mean_SCFAR)
testdata['Problem CFAR'] = testdata['Problem Name'].apply(
    lambda x: problem_correct_rate[x] if x in problem_correct_rate.keys() else mean_PCFAR)
# Add
#testdata['Unit CFAR'] = testdata['Problem Unit'].apply(lambda x: unit_correct_rate[x] if x in unit_correct_rate.keys() else mean_UCFAR)
#testdata['Section CFAR'] = testdata['Problem Section'].apply(lambda x: section_correct_rate[x] if x in section_correct_rate.keys() else mean_SCFAR)
testdata['Step CFAR'] = testdata['Step Name'].apply(
    lambda x: step_correct_rate[x] if x in step_correct_rate.keys() else mean_STCFAR)
# testdata['KC CFAR'] = testdata['KC(Default)'].apply(
#     lambda x: KC_correct_rate[x] if x in KC_correct_rate.keys() else mean_KCFAR)

#testdata['KC_length'] = testdata['KC(Default)'].astype("str").apply(lambda x: len(x))
testdata['KC_num'] = testdata['KC(Default)'].astype("str").apply(
    lambda x: 0 if x == 'nan' else (x.count('~~') + 1))

In [163]:
testdata['Step time'] = testdata['Step Name'].apply(
    lambda x: step_time[x] if x in step_time.keys() else mean_step_time)
testdata['Problem time'] = testdata['Step Name'].apply(
    lambda x: problem_time[x] if x in problem_time.keys() else mean_prob_time)

In [89]:
tmp_list = []
for i, row in testdata.iterrows():
    if pd.isnull(row['KC(Default)']):
        tmp_list.append(0.815) 
    else:
        tmp = 1
        for kc in row['KC(Default)'].split('~~'):
            if kc in KC_correct_rate.keys():
                tmp *= KC_correct_rate[kc]
            else:
                tmp *= mean_KCFAR
        tmp_list.append(tmp)
testdata['KC CFAR'] = tmp_list

tmp_list = []
for i, row in testdata.iterrows():
    if pd.isnull(row['KC(Default)']):
        tmp_list.append(0.815) 
    else:
        tmp = 0
        for kc in row['KC(Default)'].split('~~'):
            if kc in KC_correct_rate.keys():
                tmp += (KC_correct_rate[kc])
            else:
                tmp += (mean_KCFAR)
        tmp_list.append(tmp/(row['KC(Default)'].count("~")+1))
testdata['KC CFAR(avg)'] = tmp_list

In [164]:
####
# test_x
test_x = testdata[cols].copy()
test_x['Opportunity(Mean)'] = testdata['Opportunity(Default)'].astype("str").apply(
    lambda x: np.mean(list(map(int, x.replace('nan', '0').split('~~')))))
test_x['Opportunity(Min)'] = testdata['Opportunity(Default)'].astype(
    "str").apply(lambda x: min(list(map(int, x.replace('nan', '0').split('~~')))))

In [184]:
train_x.head()

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,KC CFAR(avg),Problem time,Step time,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.710197,1,1,0,0.840538,0.815000,0.815000,40.733454,49.941010,0.0,0
1,0.748749,0.710197,1,1,0,0.830609,0.815000,0.815000,40.733454,32.949725,0.0,0
2,0.748749,0.710197,1,0,1,0.966849,0.966979,0.966979,40.733454,10.225832,1.0,1
3,0.748749,0.710197,1,0,3,0.404504,0.124279,0.300044,40.733454,84.677811,1.0,1
4,0.748749,0.710197,1,1,1,0.760020,0.800770,0.800770,40.733454,34.228587,1.0,1


In [165]:
train_x.to_csv('train_pre.csv', sep='\t', index=False)
test_x.to_csv('test_pre.csv', sep='\t', index=False)

In [166]:
train_df = pd.read_csv('train_pre.csv', sep='\t')
test_df = pd.read_csv('test_pre.csv', sep='\t')

In [167]:
len(train_df[train_df['Correct First Attempt'] == 0]) / len(train_df[train_df['Correct First Attempt'] == 1])

0.28051585074659713

In [168]:
# print(test_df.shape)
# test_df = test_df.dropna(how = 'any')
# print(test_df.shape)

In [169]:
train_df.head()

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,KC CFAR(avg),Problem time,Step time,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.710197,1,1,0,0.840538,0.815000,0.815000,40.733454,49.941010,0.0,0
1,0.748749,0.710197,1,1,0,0.830609,0.815000,0.815000,40.733454,32.949725,0.0,0
2,0.748749,0.710197,1,0,1,0.966849,0.966979,0.966979,40.733454,10.225832,1.0,1
3,0.748749,0.710197,1,0,3,0.404504,0.124279,0.300044,40.733454,84.677811,1.0,1
4,0.748749,0.710197,1,1,1,0.760020,0.800770,0.800770,40.733454,34.228587,1.0,1


In [170]:
test_df.head()

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,KC CFAR(avg),Problem time,Step time,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.858006,1,1,0,0.714286,0.815000,0.815000,22.200000,22.200000,0.0,0
1,0.748749,0.766129,1,1,1,0.999435,1.000000,1.000000,17.450453,17.450453,28.0,28
2,0.748749,0.877049,1,1,1,0.622139,0.896476,0.896476,21.959627,24.643487,27.0,27
3,0.748749,0.812217,1,1,1,0.999651,1.000000,1.000000,19.727177,19.727177,65.0,65
4,0.748749,0.869446,7,1,0,0.665421,0.815000,0.815000,41.088807,41.088807,0.0,0


In [171]:
X = train_df.dropna()
y = np.array(X['Correct First Attempt']).astype(int).ravel()
del X['Correct First Attempt']
XX = test_df.dropna()
yy = np.array(XX['Correct First Attempt']).astype(int).ravel()
del XX['Correct First Attempt']

In [172]:
print(X.shape, XX.shape)

(232693, 11) (1140, 11)


In [174]:
# rf Classifier
clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=12, min_samples_split=.01,
                             n_jobs=4, random_state=None, verbose=0)
clf.fit(X, y)
print(clf.feature_importances_)
y_pred = clf.predict_proba(XX)[:,1]
y_pred_train = clf.predict_proba(X)[:,1]
print ('RandomForest', np.sqrt(mean_squared_error(y_pred, yy)))
print('RandomForest train', np.sqrt(mean_squared_error(y_pred_train, y)))

[0.025616   0.04880614 0.00119161 0.01953824 0.61985057 0.09789385
 0.08879899 0.0078693  0.08312284 0.0037748  0.00353766]
RandomForest 0.3583585863954912
RandomForest train 0.3288296472903532


In [175]:
y_pred = clf.predict(XX)
y_pred_train = clf.predict(X)
print ('RandomForest', (mean_squared_error(y_pred, yy)))
print('RandomForest train', (mean_squared_error(y_pred_train, y)))

RandomForest 0.1587719298245614
RandomForest train 0.1533393784944111


In [98]:
y_pred = (clf.predict_proba(XX)[:,1] >= 0.5)
y_pred_train = (clf.predict_proba(X)[:,1] > 0.5)
print ('RandomForest', (mean_squared_error(y_pred, yy)))
print('RandomForest train', (mean_squared_error(y_pred_train, y)))

RandomForest 0.17567567567567569
RandomForest train 0.1986216615680748


In [176]:
y_pred = clf.predict(XX)
y_pred_train = clf.predict(X)
diff = []
for i in range(len(y_pred_train)):
    if y[i] != y_pred_train[i]:
        diff.append(i)
print ('RandomForest', (mean_squared_error(y_pred, yy)))
print('RandomForest train', (mean_squared_error(y_pred_train, y)))

RandomForest 0.1587719298245614
RandomForest train 0.1533393784944111


In [180]:
diff[:50]

[2,
 9,
 10,
 11,
 15,
 17,
 24,
 28,
 31,
 36,
 38,
 39,
 40,
 41,
 50,
 51,
 53,
 56,
 66,
 69,
 72,
 77,
 83,
 91,
 95,
 111,
 114,
 130,
 135,
 138,
 139,
 173,
 177,
 181,
 182,
 189,
 195,
 197,
 200,
 207,
 209,
 212,
 231,
 232,
 242,
 251,
 258,
 263,
 269,
 282]

In [102]:
print(clf.predict_proba(X)[30:51, 1])
print(clf.predict_proba(X)[30:51, 1])
print(y[0:50] == 1)

[0.82294324 0.82388144 0.8253966  0.83528385 0.83528385 0.9115059
 0.46968255 0.8244402  0.57900277 0.58974641 0.82553915 0.82597541
 0.8265386  0.83528385 0.83528385 0.83528385 0.58974641 0.7968761
 0.7968761  0.89799909 0.4387913 ]
[0.82294324 0.82388144 0.8253966  0.83528385 0.83528385 0.9115059
 0.46968255 0.8244402  0.57900277 0.58974641 0.82553915 0.82597541
 0.8265386  0.83528385 0.83528385 0.83528385 0.58974641 0.7968761
 0.7968761  0.89799909 0.4387913 ]
[ True  True False False  True  True  True  True  True False  True False
  True  True  True  True False False  True  True  True  True  True False
 False  True  True  True  True  True  True False  True  True  True  True
  True  True False False False False  True  True  True False  True  True
  True  True]


In [103]:
train_df[30:80] #36,38,39,40, 41,50,51,53,56,
#  69,
#  72,
#  77,
#  83,
#  90,
#  91,
#  95,

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,KC CFAR(avg),Opportunity(Mean),Opportunity(Min)
30,0.748749,0.847368,1,1,1,0.755473,0.800770,0.800770,10.000000,10
31,0.748749,0.847368,1,0,1,0.694035,0.800770,0.800770,11.000000,11
32,0.748749,0.847368,1,1,1,0.707618,0.800770,0.800770,12.000000,12
33,0.748749,0.832061,1,1,0,0.840538,0.815000,0.815000,0.000000,0
34,0.748749,0.832061,1,1,0,0.830609,0.815000,0.815000,0.000000,0
35,0.748749,0.832061,1,1,1,0.966849,0.966979,0.966979,4.000000,4
36,0.748749,0.832061,1,1,3,0.404504,0.124279,0.300044,6.000000,4
37,0.748749,0.832061,1,1,1,0.760020,0.800770,0.800770,13.000000,13
38,0.748749,0.832061,1,0,3,0.708202,0.184001,0.344642,6.000000,2
39,0.748749,0.832061,1,0,3,0.737218,0.184001,0.344642,7.000000,3


In [380]:
testdata[14:23]

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Hints,Corrects,KC(Default),Opportunity(Default),Problem Unit,Problem Section,Personal CFAR,Problem CFAR,KC CFAR,KC_num
14,21616,8o0mJnIpFH,"Unit CTA1_06, Section CTA1_06-3",JAN05,2,R5C1,NaN,NaN,NaN,NaN,...,NaN,NaN,Using simple numbers~~Using large numbers~~Fin...,24~~15~~18,Unit CTA1_06,Section CTA1_06-3,1344,0.777679,0.686542,3
15,22178,8o0mJnIpFH,"Unit ES_03, Section ES_03-6",EG51A,2,constant termsR2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Unit ES_03,Section ES_03-6,1344,0.885030,0.559564,0
16,22320,8o0mJnIpFH,"Unit CTA1_08, Section CTA1_08-2",FEB03,1,yMin,NaN,NaN,NaN,NaN,...,NaN,NaN,Changing axis bounds,98,Unit CTA1_08,Section CTA1_08-2,1344,0.763982,0.775749,1
17,22862,8o0mJnIpFH,"Unit ES_04, Section ES_04-15",LIT58,1,7+y*(b+n) = r+f,NaN,NaN,NaN,NaN,...,NaN,NaN,[SkillRule: Eliminate Parens; {CLT nested; CLT...,70,Unit ES_04,Section ES_04-15,1344,0.741007,0.865864,1
18,23277,8o0mJnIpFH,"Unit CTA1_10, Section CTA1_10-5",DISTFB08_SP,1,R3C2,NaN,NaN,NaN,NaN,...,NaN,NaN,"Using simple numbers~~Write expression, positi...",78~~45~~56,Unit CTA1_10,Section CTA1_10-5,1344,0.806701,0.460606,3
19,23522,8o0mJnIpFH,"Unit ES_07, Section ES_07-4",LIT63A,1,n*t-h*r = c*j+z*n,NaN,NaN,NaN,NaN,...,NaN,NaN,"[SkillRule: Remove constant; {ax+b=c, positive...",228,Unit ES_07,Section ES_07-4,1344,0.821018,0.787275,1
20,23621,8o0mJnIpFH,"Unit CTA1_12, Section CTA1_12-2",SYS03,1,R3C3,NaN,NaN,NaN,NaN,...,NaN,NaN,Using simple numbers~~Using small numbers~~Wri...,94~~104~~55,Unit CTA1_12,Section CTA1_12-2,1344,0.667472,0.518980,3
21,35682,g209g5Vve6,"Unit CTA1_08, Section CTA1_08-3",REAL37,1,Slope Field 1,NaN,NaN,NaN,NaN,...,NaN,NaN,Entering the slope,20,Unit CTA1_08,Section CTA1_08-3,2579,0.823810,0.787045,1
22,36252,g209g5Vve6,"Unit ES_04, Section ES_04-14",LIT23A,1,(n*j+56)/n = r/n,NaN,NaN,NaN,NaN,...,NaN,NaN,[SkillRule: Multiply/Divide; [Typein Skill: {R...,28,Unit ES_04,Section ES_04-14,2579,0.784946,0.912086,1


In [301]:
X.head()

,Personal CFAR,Problem CFAR,Problem View,KC_num,KC CFAR,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.710197,1,0,0.559564,0.0,0
1,0.748749,0.710197,1,0,0.559564,0.0,0
2,0.748749,0.710197,1,1,0.966979,1.0,1
3,0.748749,0.710197,1,3,0.443541,1.0,1
4,0.748749,0.710197,1,1,0.817953,1.0,1


In [185]:
print(y_pred)

[0.70464794 0.90034861 0.83940296 0.84467888 0.49334496 0.85447626
 0.49517352 0.86528865 0.10559067 0.92120628 0.96245791 0.90367308
 0.87289405 0.89071147 0.98253456 0.8646177  0.67579006 0.97146262
 0.85377044 0.77064164 0.92158692 0.8646177  0.68108584 0.60320665
 0.96886464 0.93483043 0.88320112 0.87838016 0.70738053 0.86679648
 0.70158135 0.86834382 0.40540573 0.92732408 0.8915245  0.84623782
 0.93000898 0.52922163 0.91979022 0.94271517 0.8170657  0.80453617
 0.93609541 0.92456514 0.87304289 0.48123825 0.68104276 0.9707783
 0.52903393 0.85638998 0.87555089 0.7498415  0.39065227 0.89530165
 0.75886506 0.62971913 0.92738666 0.86550894 0.59002595 0.89933063
 0.87615004 0.75660832 0.87405193 0.66418015 0.97215128 0.81643729
 0.87918949 0.91536172 0.95444488 0.96747534 0.90900121 0.34462517
 0.87520704 0.8565948  0.89598835 0.83008866 0.74593356 0.5292577
 0.99202433 0.81035389 0.40880319 0.95548772 0.74242421 0.83248034
 0.90379095 0.45440824 0.67411262 0.97904026 0.49805623 0.849232

In [32]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=.01,
                             n_jobs=4, random_state=None, verbose=0)
# clf.fit(X, y)
score_pre = cross_val_score(clf, X, y, cv=10).mean()

In [250]:
score_pre

0.8458693081172713

In [253]:
clf.predict(X)

array([1, 1, 1, ..., 1, 0, 1])

In [125]:
import matplotlib.pyplot as plt

In [232]:
scorel = []
for i in [9, 10, 11, 12, 13, 14]:
    print(i)
    rfc = RandomForestClassifier(n_estimators=100,
                                 n_jobs=-1,
                                 random_state=90,
                                 max_depth=i)
    score = cross_val_score(rfc, X, y,cv=10).mean()
    scorel.append(score)
# print(max(scorel),(scorel.index(max(scorel))*10)+1)
# plt.figure(figsize=[20,5])
# plt.plot(range(1,201,10),scorel)
# plt.show()

9
10
11
12
13
14


In [233]:
scorel

[0.8487566142181219,
 0.849671763838573,
 0.8506041122029468,
 0.8508017408453185,
 0.8501658349829938,
 0.8500541105276727]

In [229]:
scorel

[0.8393814112122644,
 0.8436651906715931,
 0.8477898790728894,
 0.8506041122029468]

In [164]:
X

,Personal CFAR,Problem CFAR,Problem View,KC_num,Step CFAR,KC CFAR,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.710197,1,0,0.840631,0.559564,0.0,0
1,0.748749,0.710197,1,0,0.830699,0.559564,0.0,0
2,0.748749,0.710197,1,1,0.966979,0.966979,1.0,1
3,0.748749,0.710197,1,3,0.404477,0.443541,1.0,1
4,0.748749,0.710197,1,1,0.760107,0.817953,1.0,1
...,...,...,...,...,...,...,...,...
232739,0.733333,0.776978,1,2,0.755559,0.572193,11.0,1
232740,0.733333,0.776978,1,0,0.830699,0.559564,0.0,0
232741,0.733333,0.776978,1,1,0.851036,0.796004,7.0,7
232742,0.733333,0.776978,1,3,0.404477,0.443541,8.0,5


In [163]:
train_df.head()

,Personal CFAR,Problem CFAR,Problem View,Correct First Attempt,KC_num,Step CFAR,KC CFAR,Opportunity(Mean),Opportunity(Min)
0,0.748749,0.710197,1,1,0,0.840631,0.559564,0.0,0
1,0.748749,0.710197,1,1,0,0.830699,0.559564,0.0,0
2,0.748749,0.710197,1,0,1,0.966979,0.966979,1.0,1
3,0.748749,0.710197,1,0,3,0.404477,0.443541,1.0,1
4,0.748749,0.710197,1,1,1,0.760107,0.817953,1.0,1


In [455]:
## Adaboost
clf = AdaBoostRegressor(base_estimator=None, n_estimators=50,
                        learning_rate=1.0, loss='exponential',
                        random_state=None)
clf.fit(X, y)
y_pred = clf.predict(XX)
# print clf.best_estimator_
print ('AdaBoost', np.sqrt(mean_squared_error(y_pred, yy)))

AdaBoost 0.4022577513799062


In [459]:
##  Gradient Decision Tree
clf = GradientBoostingClassifier(n_estimators=200)
clf.fit(X, y)
y_pred = clf.predict_proba(XX)[:, 1]
#print (clf.best_estimator_)
print ('GBDTClassifier', np.sqrt(mean_squared_error(y_pred, yy)))

GBDTClassifier 0.35528424518488205


In [460]:
clf.feature_importances_

array([0.03415561, 0.03488461, 0.00139682, 0.00143972, 0.85860611,
       0.06210847, 0.00317993, 0.00422874])

In [461]:
## Logistic Regression
clf = LogisticRegression(penalty='l2')
clf.fit(X, y)
y_pred = clf.predict_proba(XX)[:, 1]
#print (clf.best_estimator_)
print ('LogicalClassifier', np.sqrt(mean_squared_error(y_pred, yy)))

LogicalClassifier 0.36332021103646334


/Users/zhengyuting/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [462]:
clf.feature_importances_

AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

In [151]:
train_df = pd.read_csv('train_pre.csv', sep='\t')
test_df = pd.read_csv('test_pre.csv', sep='\t')

In [152]:
X = train_df.dropna()
y = np.array(X['Correct First Attempt']).astype(int).ravel()
del X['Correct First Attempt']
XX = test_df
yy = np.array(XX['Correct First Attempt']).astype(float).ravel()
del XX['Correct First Attempt']

In [153]:
test_df.shape

(1140, 8)

In [154]:
clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=.01,
                                     n_jobs=4, random_state=None, verbose=0)
clf.fit(X, y)
y_pred = clf.predict_proba(XX)[:, 1]

In [155]:
for index, val in enumerate(yy):
    if np.isnan(val):
        yy[index] = y_pred[index]

In [158]:
test_df.shape

(1140, 8)

In [161]:
yy[:50]

array([0.89273528, 0.98985909, 0.92387302, 0.99123919, 0.77079667,
       0.91872856, 0.39559033, 0.86767413, 0.92602309, 0.91014879,
       0.89029571, 0.79518089, 0.96343833, 0.88215236, 0.        ,
       1.        , 1.        , 1.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 1.        , 1.        ,
       1.        , 0.        , 1.        , 1.        , 1.        ,
       0.        , 1.        , 1.        , 0.        , 0.81382866,
       0.89591608, 0.89461552, 0.91169137, 0.65994123, 0.89757478,
       0.92482836, 0.80632635, 0.89283023, 0.90815411, 0.72314761,
       0.91702249, 0.95441284, 0.82024888, 0.9188468 , 0.83291441])

In [142]:
test = pd.read_csv('data/test.csv', sep='\t')

In [143]:
test.head()

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,10039,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP01,1,2/300*X = 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10482,52vEY7f17k,"Unit CTA1_06, Section CTA1_06-3",FOR05,1,xScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,28
2,11024,52vEY7f17k,"Unit ES_03, Section ES_03-6",EG52,1,2.9 = -2x+3.7+x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SkillRule: Consolidate vars, no coeff; CLT]",27
3,11476,52vEY7f17k,"Unit CTA1_08, Section CTA1_08-3",REAL10,1,yScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,65
4,11948,52vEY7f17k,"Unit ES_04, Section ES_04-12",EG58,7,xR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [321]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
# cols = ['Personal CFAR', 'Problem CFAR', 'Anon Student Id', 'Problem Unit', 'Problem Section',\
#         'Step Name', 'Problem Name', 'Problem View', 'Correct First Attempt', 'KC_num', 'Step CFAR', 'KC CFAR']
cols = ['Personal CFAR', 'Problem CFAR',\
        'Problem View', 'Correct First Attempt', 'KC_num', 'KC CFAR']


traindata = pd.read_csv(train_file, sep='\t')

In [310]:
KC_correct_rate = {}
for KC, group in traindata.groupby(['KC(Default)']):
    if not pd.isnull(KC):  # KC != 'nan':
        KC_correct_rate[KC] = 1.0 * len(group[group['Correct First Attempt'] == 1]) / len(
            group['Correct First Attempt'])

In [312]:
for _, row in traindata.iterrows():
    split_KC = []
    if pd.isnull(row['KC(Default)']):
        continue
    split_KC
    break

0 Row                                                       9938
Anon Student Id                                     52vEY7f17k
Problem Hierarchy              Unit CTA1_13, Section CTA1_13-1
Problem Name                                            PROP04
Problem View                                                 1
Step Name                                                 R1C1
Step Start Time                          2005-09-09 12:23:34.0
First Transaction Time                   2005-09-09 12:24:07.0
Correct Transaction Time                 2005-09-09 12:24:07.0
Step End Time                            2005-09-09 12:24:07.0
Step Duration (sec)                                         33
Correct Step Duration (sec)                                 33
Error Step Duration (sec)                                  NaN
Correct First Attempt                                        1
Incorrects                                                   0
Hints                                                

In [320]:
traindata['Personal CFAR'] = (testdata['Anon Student Id']) #testdata['Anon Student Id']#.apply(lambda x: student_correct_rate[x][0] if x in student_correct_rate.keys() else mean_SCFAR)

In [316]:
traindata.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)',
       'Personal CFAR'],
      dtype='object')

In [ ]:
def get_kc_dummies(row, headers, col):
    tokens = row[col].split("~~")
    opps = np.asarray([int(s) if s.lower() != "nan" else 0 for s in row["Opportunity(Default)"].split("~~")])
    opps = np.log(opps + 1)
    sr = pd.Series(np.zeros(len(headers)), index = headers)
    sr[tokens] = opps
    return sr.to_sparse(fill_value=0)

In [ ]:
compound_KCs = set(df["KC(Default)"])
print("KC(Default) has %d compound values." % len(compound_KCs))
KCs = set()
for s in compound_KCs:
    KCs = KCs.union(set(s.split("~~")))

print("There are %d atomic KCs, so we will be adding as many columns to this dataframe" % len(KCs))

# Cast and split columns
kc_features = df.apply(get_kc_dummies, axis=1, result_type="expand", args=(KCs, "KC(Default)"))
kc_features.head()

In [308]:
print(len(student_correct_rate),len(problem_correct_rate),len(step_correct_rate), len(KC_correct_rate))

174 1021 60709 348


In [314]:
student_correct_rate['02i5jCrfQK']

(5171, 6654)